In [1]:
import torch
import dlc_practical_prologue as prologue
from importlib import reload
reload(prologue)
from dlc_practical_prologue import *
import Module, modules, optimizers, helpers
reload(Module)
reload(modules)
reload(helpers)
from modules import *
from helpers import *
from optimizers import *
torch.set_grad_enabled(False)

#torch.set_default_tensor_type(torch.DoubleTensor)

def test_model(model, test_input, test_target):
    test_output = model(test_input)
    output_to_prediction = torch.ge(test_output, 0.5).flatten()
    nb_correct = torch.sum(output_to_prediction == test_target.type(torch.ByteTensor)).item()
    acc_pairs = nb_correct / len(test_input)
    return acc_pairs

train_input, train_target, train_classes, test_input, test_target, test_classes = \
       load_random_datasets()

nb_hidden1 = 100
nb_hidden2 = 40

train_input = train_input.view(len(train_input), -1)
train_target = train_target.type(torch.FloatTensor)
test_input = test_input.view(len(train_input), -1)
test_target = test_target.type(torch.FloatTensor)

#train_input = train_input.view(len(train_input), -1).type(torch.DoubleTensor)
#train_target = train_target.type(torch.DoubleTensor)
#test_input = test_input.view(len(train_input), -1).type(torch.DoubleTensor)
#test_target = test_target.type(torch.DoubleTensor)

input_size = train_input.shape[1] # 392
nb_epochs = 300

model_mnist = Sequential(
    Linear('fc1', input_size, nb_hidden1), ReLU(),
    Linear('fc2', nb_hidden1, nb_hidden2), Tanh(),
    Linear('fc3', nb_hidden2, 1))

from torch import nn
model_mnist_torch = nn.Sequential(
    nn.Linear(input_size, nb_hidden1), nn.ReLU(),
    nn.Linear(nb_hidden1, nb_hidden2), nn.Tanh(),
    nn.Linear(nb_hidden2, 1))
set_initial_parameters(model_mnist, model_mnist_torch)
del nn


lr = 0.01
criterion_mnist = MSELoss()
optimizer_mnist = SGD(model_mnist, lr=lr)
loss_history_mnist = []
test_acc_history_mnist = []
nb_epochs = 300
for e in range(nb_epochs):
    optimizer_mnist.zero_grad()
    
    output_mnist = model_mnist(train_input) # torch.Size([1000, 1])
    
    loss_mnist = criterion_mnist(output_mnist.flatten(), train_target)
    loss_history_mnist.append(loss_mnist)
    
    grad_output_mnist = criterion_mnist.backward().view(-1,1)
    test_acc_mnist = test_model(model_mnist, test_input, test_target)
    test_acc_history_mnist.append(test_acc_mnist)
    if e % 40 == 0: print("e = {}, test accuracy = {}, loss = {}".format(e, test_acc_mnist, loss_mnist))
    #print("e = {}, test accuracy = {}, loss = {}".format(e, test_acc_mnist, loss_mnist))
    model_mnist.backward(grad_output_mnist)
    optimizer_mnist.step(loss_mnist)

print()
from torch import nn
torch.set_grad_enabled(True)

criterion_mnist_torch = nn.MSELoss()
optimizer_mnist_torch = torch.optim.SGD(model_mnist_torch.parameters(), lr=lr)
loss_history_mnist_torch = []
test_acc_history_mnist_torch = []

for e in range(nb_epochs):
    optimizer_mnist_torch.zero_grad()
    output_mnist_torch = model_mnist_torch(train_input)
    #print("output = ", output_mnist_torch[:5])
    #print()
    loss_mnist_torch = criterion_mnist_torch(output_mnist_torch.flatten(), train_target)
    loss_history_mnist_torch.append(loss_mnist_torch)
    test_acc_mnist_torch = test_model(model_mnist_torch, test_input, test_target)
    test_acc_history_mnist_torch.append(test_acc_mnist_torch)
    if e % 40 == 0: print("e = {}, test accuracy = {}, loss = {}".format(e, test_acc_mnist_torch, loss_mnist_torch))
    #print("e = {}, test accuracy = {}, loss = {}".format(e, test_acc_mnist_torch, loss_mnist_torch))
    loss_mnist_torch.backward()
    optimizer_mnist_torch.step()
    
del nn
torch.set_grad_enabled(False)

e = 0, test accuracy = 0.435, loss = 0.5342389345169067
e = 40, test accuracy = 0.44, loss = 0.376302570104599
e = 80, test accuracy = 0.453, loss = 0.41157200932502747
e = 120, test accuracy = 0.472, loss = 0.5179829597473145
e = 160, test accuracy = 0.503, loss = 0.8302551507949829
e = 200, test accuracy = 0.537, loss = 4.003942966461182
e = 240, test accuracy = 0.534, loss = 12.101386070251465
e = 280, test accuracy = 0.517, loss = 16.579118728637695

e = 0, test accuracy = 0.435, loss = 0.5342389345169067
e = 40, test accuracy = 0.683, loss = 0.1847561001777649
e = 80, test accuracy = 0.725, loss = 0.1547391265630722
e = 120, test accuracy = 0.739, loss = 0.1388862580060959
e = 160, test accuracy = 0.742, loss = 0.12781952321529388
e = 200, test accuracy = 0.748, loss = 0.11909683793783188
e = 240, test accuracy = 0.757, loss = 0.11175363510847092
e = 280, test accuracy = 0.756, loss = 0.10531888902187347


In [2]:
import matplotlib.pyplot as plt
plt.plot(loss_history_mnist, label='Custom DL framework')
plt.plot(loss_history_mnist_torch, label='PyTorch')
plt.legend()
plt.show()


<Figure size 640x480 with 1 Axes>